In [18]:
!pip install --upgrade tf_slim

import numpy.random as rnd
import numpy as np
import tensorflow as tf
import tf_slim as slim
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.preprocessing import label_binarize

In [22]:
discount_rate = 0.95 # gamma
learning_rate0 = 0.05
learning_rate_decay = 0.1
n_iterations = 1000

# Input Layer - número dos estados de entrada
# Output Layer - número de ações no ambiente

state = label_binarize([0, 1, 2, 3], classes=[0, 1, 2, 3])
print("Matriz dos Estados:")
print(state, "\n")

state_size  = state.shape[0]
action_size = 4

print("Numero de Estados:", state_size)
print("Numero de Acoes:", action_size)

model = keras.Sequential()
model.add(keras.layers.Dense(state_size, input_shape=(None,state_size), activation='relu'))
model.add(keras.layers.Dense(action_size, activation='softmax'))
model.compile(optimizer=tf.optimizers.Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Matriz dos Estados:
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]] 

Numero de Estados: 4
Numero de Acoes: 4


In [24]:
numTimeSlots = 1000
epsilon = 0.5 

for iteration in range(numTimeSlots):
    
    print("Time Slot   :", iteration)
    print("----------------------")
    
    # Estado Atual
    c_state = state[iteration % 4,:] # seleciona um estado por vez (uma linha por vez)
    print("Estado atual:", c_state)
        
    # Escolha da Acao
    if np.random.uniform() < epsilon:
        predicted = model.predict(c_state.reshape(1,4))
        action = np.argmax(predicted[0])
    else:
        action = np.random.randint(0, state_size)
    print("Acao        :", action)
     
    # Proximo Estado - apenas para representacao
    next_s = state[(iteration + 1) % 4] # seleciona uma linha a seguir +1 por vez
    print("Next State  :", next_s)
     
    s_index = np.argwhere(c_state==1)[0][0]
    #print("indice do estado atual", s_index)
    
    n_s_index = np.argwhere(next_s)[0][0]
    #print("indice do proximo estado", n_s_index, "\n")
    
    #Reward
    if action == n_s_index:
        reward = -1.0
    else:
        reward = +1.0
    print("Reward      :", reward)
    print("-------------", "\n")

    # Train
    target = reward
    if iteration != numTimeSlots-1:
      predicted = model.predict(next_s.reshape(1,4))[0]
      target = (reward + discount_rate * np.amax(predicted))

    target_f = model.predict(c_state.reshape(1,4))
    target_f[0][action] = target
    model.fit(c_state.reshape(1,4), target_f, epochs=1, verbose=0)

Streaming output truncated to the last 5000 lines.
Time Slot   : 375
----------------------
Estado atual: [0 0 0 1]
Acao        : 3
Next State  : [1 0 0 0]
Reward      : 1.0
------------- 

Time Slot   : 376
----------------------
Estado atual: [1 0 0 0]
Acao        : 3
Next State  : [0 1 0 0]
Reward      : 1.0
------------- 

Time Slot   : 377
----------------------
Estado atual: [0 1 0 0]
Acao        : 1
Next State  : [0 0 1 0]
Reward      : 1.0
------------- 

Time Slot   : 378
----------------------
Estado atual: [0 0 1 0]
Acao        : 1
Next State  : [0 0 0 1]
Reward      : 1.0
------------- 

Time Slot   : 379
----------------------
Estado atual: [0 0 0 1]
Acao        : 2
Next State  : [1 0 0 0]
Reward      : 1.0
------------- 

Time Slot   : 380
----------------------
Estado atual: [1 0 0 0]
Acao        : 0
Next State  : [0 1 0 0]
Reward      : 1.0
------------- 

Time Slot   : 381
----------------------
Estado atual: [0 1 0 0]
Acao        : 0
Next State  : [0 0 1 0]
Reward    

In [25]:
 for idx in range(state_size):

    c_state = state[idx]
    print("Estado atual:", c_state)
 
    q = model.predict(c_state.reshape(1,4))
    print('Q:',q[0])
    print('-----\n')

Estado atual: [1 0 0 0]
Q: [0.01037942 0.59799933 0.00782561 0.38379562]
-----

Estado atual: [0 1 0 0]
Q: [7.0036862e-05 9.9966574e-01 1.4640043e-04 1.1773953e-04]
-----

Estado atual: [0 0 1 0]
Q: [6.2057056e-06 9.9996793e-01 1.7311879e-05 8.6323835e-06]
-----

Estado atual: [0 0 0 1]
Q: [1.8696504e-05 9.9990141e-01 5.2261508e-05 2.7707480e-05]
-----



In [ ]:
print("Com base na matriz gerada no Time Slot = 99, podemos observar que:\n")
print("O Canal Ch0 executa a Acao a2, que possui Q-value=", Q[0][2], "\n")
print("O Canal Ch1 executa a Acao a3, que possui Q-value=", Q[1][3], "\n")
print("O Canal Ch2 executa a Acao a0, que possui Q-value=", Q[2][0], "\n")
print("O Canal Ch3 executa a Acao a1, que possui Q-value=", Q[3][1])